<a href="https://colab.research.google.com/github/leoalexandre/mvp_adbp/blob/main/mvp_anp_seismic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP de Análise de Dados e Boas Práticas
Por Leonardo da Silva Alexandre

## 1.   Definição do Problema

O dataset utilizado neste projeto foi retirado do site da Agência Nacional do Petróleo (ANP). A criação veio a partir da tabela de atributos retirada do shapefile de todos os programas sísmicos gerados pelo mercado de óleo e gás brasileiro, somado a aplicação de algumas regras presentes na legislação brasileira gerida pela ANP. A legislação nacional pode ser encontrada dentro dos sites da ANP. O shapefile está alocado mais precisamente em:

https://www.gov.br/anp/pt-br/assuntos/exploracao-e-producao-de-oleo-e-gas/dados-tecnicos/acervo-de-dados

O dataset apresenta atributos relacionados aos programas sísmicos com relação a datas, localização, volume, valor, empresas proprietárias dos dados e período de confidencialidade. Seu objetivo é vislumbrar novos processamentos ou aquisições sísmicas com maior potencial de interesse do mercado a partir da identificação de uma ausência de dados de processamento e suas versões em áreas das bacias sedimentares brasileiras que apresentem maior visibilidade da indústria de óleo e gás.
Informações sobre atributos:

  1. **VOLUME_GB** – Volume da soma dos arquivos do projeto sísmico;
  2. **VALOR_UD** – Valor em UD do projeto sísmico;
  3. **PROJETO** – Projeto sísmico oriundo de um programa sísmico;
  4. **TIPO_PROCESSAMENTO** – Tipo de processamento sísmico realizado no programa sísmico;
  5. **PUBLICIDADE** – Classificação do programa sísmico em público ou confidencial;
  6. **PROGRAMA** – Programa sísmico criado pela ANP por solicitação da empresa;
  7. **TIPO_PROGRAMA** – Classificação do programa sísmico em Exclusivo (Pré-98), Exclusivo, Não-Exclusivo ou de fomento;
  8. **TECNOLOGIA** – Dimensionalidade do dado sísmico;
  9. **INÍCIO** – Data de início do programa;
  10. **TÉRMINO** – Data de término do programa;
  11. **DATA_PUBLICIDADE** – Data na qual o programa se torna público perante a legislação vigente;
  12. **BLOCO** – Bloco exploratório onde está localizado o programa sísmico;
  13. **CAMPO** – Campo de exploração onde está localizado o programa sísmico;
  14. **BACIA** – Bacia geológica onde está localizado o programa sísmico;
  15. **EAD** – Empresa de Aquisição de Dados responsável pelo programa sísmico;
  16. **OPERADORA** – Empresa Operadora detentora dos direitos sobre os dados do programa sísmico.

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Carga de Dados

Iremos usar o pacote Pandas (Python Data Analysis Library) para carregar de um arquivo .csv sem cabeçalho disponível online.

Com o dataset carregado, iremos explorá-lo um pouco.

MUDAR!!!!!!

In [45]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/leoalexandre/mvp_adbp/main/dados_anp.csv"

# Informa o cabeçalho das colunas
colunas = ['Vol_GB', 'UDs', 'Projeto', 'Type_Proc', 'Pub', 'Programa', 'Type_Prog', 'Tec', 'date_start','date_end', 'date_pub', 'Bloco', 'Campo', 'Bacia', 'EAD', 'Operator']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')


In [46]:
dataset.head()

,Vol_GB,UDs,Projeto,Type_Proc,Pub,Programa,Type_Prog,Tec,date_start,date_end,date_pub,Bloco,Campo,Bacia,EAD,Operator
0,0.016620,1,0007_SERGIPE_ALAGOAS_39,STK FIN,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
1,NaN,,0007_SERGIPE_ALAGOAS_39,Field Data,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
2,14.677270,2.935453938,0022_BRASILSPAN01_PSTM,MIG PSTM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
3,18.291451,3.658290235,0022_BRASILSPAN01_PSDM,MIG PSDM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
4,4283.437811,42.83437811,0022_BRASILSPAN01,Field Data,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
